## Smishing detection using [NousResearch/Meta-Llama-3-8B-Instruct](https://huggingface.co/NousResearch/Meta-Llama-3-8B-Instruct) - evaluation of the model

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import pickle
import torch

# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# test data
with open("./data/test_data.pkl", "rb") as input_file:
    test_data = pickle.load(input_file)

X_test = test_data["X_test"]
y_test = test_data["y_test"]

total_hams_count = 0
total_smishes_count = 0

for label in y_test:
    if label == "ham":
        total_hams_count += 1
    if label == "smish":
        total_smishes_count += 1

print("There is {} hams and {} smishes in the test dataset.".format(total_hams_count, total_smishes_count))

There is 954 hams and 161 smishes in the test dataset.


In [3]:
# Model name
model_name = 'NousResearch/Meta-Llama-3-8B-Instruct'

# Load the model
model = AutoModelForCausalLM.from_pretrained(model_name)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
false_hams_indicies = []
false_smishes_indicies = []
false_hams_count = 0
false_smishes_count = 0
true_hams_count = 0
true_smishes_count = 0
errors_count = 0
errors_indicies = []
errors = []


for i in range(len(X_test)):
    prompt = (
    "<|begin_of_text|><|start_header_id|>user<|end_header_id|>"
    "Do you think it is a ham or smish message? "
    "Your output should be a single word 'smish' or 'ham'. "
    "Do not write a sentence. "
    "Output is case-sensitive. "
    "SMS content: {}"
    "<|eot_id|><|start_header_id|>assistant<|end_header_id|>"
    ).format(X_test[i])

    pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=500)
    answer = pipe(prompt)
    answer = answer[0]['generated_text'].split("<|start_header_id|>assistant<|end_header_id|>")[1].lower().strip()

    if answer not in ["ham", "smish"]:
        errors_count += 1
        errors_indicies.append(i)
        errors.append(answer)
        continue
    elif answer == "ham" and y_test[i] == "ham": # correctly recognized as a ham
        true_hams_count += 1
    elif answer == "smish" and y_test[i] == "smish": # correctly recognized as a smish
        true_smishes_count += 1
    elif answer == "ham" and y_test[i] == "smish": # wrongly recognized as a ham
        false_hams_indicies.append(i)
        false_hams_count += 1
    elif answer == "smish" and y_test[i] == "ham": # wrongly recognized as a smish
        false_smishes_indicies.append(i)
        false_smishes_count += 1

# errors warning
if errors_count != 0:
    if errors_count == 1:
        print("WARNING: {} error".format(errors_count))
    else:
        print("WARNING: {} errors".format(errors_count))

# save results for further analysis
results = {"FN" : false_hams_count, "FP" : false_smishes_count,
           "TN" : true_hams_count, "TP" : true_smishes_count,
           "FN_indicies" : false_hams_indicies, "FP_indicies" : false_smishes_indicies,
            "errors_count" : errors_count, "errors" : errors, "errors_indicies" : errors_indicies}

with open("./results/results_llama_3_8b.pkl", 'wb') as handle:
    pickle.dump(results, handle, protocol=pickle.HIGHEST_PROTOCOL)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:12

### The performance of the base Llama 3 8b model in smishing detection.

In [5]:
FN = false_hams_count     #FN - messages wrongly recognized as not smishes (hams)
FP = false_smishes_count  #FP - messages wrongly recognized as smishes
TN = true_hams_count      #TN - messages correctly recognized as not smishes (hams)
TP = true_smishes_count   #TP - messages correctly recognized as smishes
TOTAL = FN + FP + TN + TP

In [6]:
print("Messages wrongly recognized as hams: {0:.2f}%".format(FN / TOTAL * 100))
print("Messages wrongly recognized as smishes: {0:.2f}%".format(FP / TOTAL * 100))
print("Messages correctly recognized as hams: {0:.2f}%".format(TN / TOTAL * 100))
print("Messages correctly recognized as smishes: {0:.2f}%".format(TP / TOTAL * 100))

Messages wrongly recognized as hams: 9.87%
Messages wrongly recognized as smishes: 57.63%
Messages correctly recognized as hams: 27.92%
Messages correctly recognized as smishes: 4.58%


#### Accuracy

In [7]:
accuracy = (TP + TN) / TOTAL
print("{0:.2f}%".format(accuracy * 100))

32.50%


#### Recall

In [8]:
recall = TP / (TP + FN)
print("{0:.2f}%".format(recall * 100))

31.68%


#### Precision

In [9]:
precision = TP / (TP + FP)
print("{0:.2f}%".format(precision * 100))

7.36%


#### F1 score

In [10]:
F1_score = TP / (TP + (FP + FN) / 2)
print("{0:.2f}%".format(F1_score * 100))

11.94%
